# Question 1

In [1]:
# 2)
import numpy as np
# a)
sample = np.array([4, 7, 9, 11, 15])
var = np.var(sample, ddof=1)  # use ddof of 1 because sample
print(f'Variance for a): {var}')

Variance for a): 17.2


In [2]:
# b)
sample = np.array([18, 22, 25, 30, 35, 40])
var = np.var(sample, ddof=1)  # use ddof of 1 because sample
print(f'Variance for b): {var}')

Variance for b): 68.26666666666668


In [3]:
# 4)

# a)
sample = np.array([12, 14, 16, 18, 20])
var = np.var(sample, ddof=0)  # use ddof 0 for population
print(f'Variance for a): {var}')

Variance for a): 8.0


In [4]:
# b)
sample = np.array([28, 32, 36, 40, 44, 48])
var = np.var(sample, ddof=0)  # use ddof 0 for population
print(f'Variance for b): {var}')

Variance for b): 46.666666666666664


5)
a): They key difference between empirical and population variance calculations is when calculation the empirical variance, the denominator used is n-1 while for the population variance the denominator used is n <br>
b): An example of where you would want to use population variance over empirical variance is for example if you wanted to find the variance of the ages of kids in a specific class. However, if you wanted to find the variance in ages for an entire school, it would be better to take a sample of kids' ages and then use the empirical variance in your calculation.

# Question 2

In [5]:
import numpy as np
eta = 0.5

X = np.array([[1, -2, 0, 1], [-2, -1, 1, 2], [1, 2, -1, 1]])
w0 = np.array([[-1], [0], [1], [1]])
Y = np.array([[2], [3], [-1]])

n = X.shape[0]

def f_prime(w, lambda_value):
    return ((2/n)*(X.T.dot(X.dot(w)-Y)) + 2 * lambda_value * w)

w = w0.copy()
for i in range(2):
    w = w - eta * f_prime(w, 2)

print(f'W after one steps of gradient descent: {w}')

W after one steps of gradient descent: [[-13.        ]
 [ -5.11111111]
 [  7.        ]
 [  9.44444444]]


# Question 3

In [6]:
import numpy as np
from numpy import genfromtxt

X = genfromtxt('stock_prediction_data.csv', delimiter=',')

y = genfromtxt('stock_price.csv', delimiter=',')
y = np.reshape(y, (len(y),1))

In [7]:
# Split data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.2)
X_test, X_val, y_test, y_val = train_test_split(X_rest, y_rest, test_size=0.5)

# Scale data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Create second degree polynomial feature map
X_train = PolynomialFeatures(degree=2).fit_transform(X_train)
X_val = PolynomialFeatures(degree=2).fit_transform(X_val)
X_test = PolynomialFeatures(degree=2).fit_transform(X_test)

Lasso Constraint

In [8]:
def f_prime(phi, w, y, n, lambda1):
    return (2/n) * phi.T.dot(phi.dot(w) - y) + lambda1 * np.sign(w)

def mse_function(y_pred, y):
    return np.mean((y_pred- y)**2)

def gradient_descent(phi, y, num_iterations, coef, lambda1):
    n, m = phi.shape

    w = np.random.rand(m, 1)  # Generate initial w

    for i in range(num_iterations):
        w -= coef * f_prime(phi, w, y, n, lambda1)

    # MSE for final interation
    final_mse = mse_function(phi.dot(w), y)

    return w, final_mse

In [9]:
# Find MSE for sklearn and gradient descent
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=1)
lasso.fit(X_train, y_train.flatten())
y_hat = lasso.predict(X_train)
y_hat = y_hat.reshape(-1, 1)

sklearn_mse = mse_function(y_hat, y_train)
print(f'sklearn mse for lasso: {sklearn_mse}')

w, mse = gradient_descent(X_train, y_train, 10000, 0.02, 1)
print(f'Gradient descent mse for lasso: {mse}')

sklearn mse for lasso: 6.607796083797113
Gradient descent mse for lasso: 2.0322905782354983


In [18]:
# identify best lambda value
values = np.logspace(-4, 1, 10) 

best_value = values[0] # initialize best value
best_mse = float('inf') # Store intiial mse as very high value

for lambda_value in values:
    # Get predicted w for lambda
    w, mse = gradient_descent(X_train, y_train, 10000, 0.02, lambda_value)
    y_hat = X_val.dot(w)

    # get MSE for predictede w
    val_mse = mse_function(y_hat, y_val)

    if best_mse > val_mse:
        best_mse = val_mse
        best_value = lambda_value 

print(f'Best lambda value: {best_value} with mse: {best_mse}')

Best lambda value: 0.016681005372000592 with mse: 0.0631533753696244


Ridge constraint

In [11]:
def f_prime_ridge(phi, w, y, n, lambda1):
    return (2/n) * phi.T.dot(phi.dot(w) - y) + lambda1 * (2*w)

def mse_function(y_pred, y):
    return np.mean((y_pred- y)**2)

def gradient_descent_ridge(phi, y, num_iterations, coef, lambda1):
    n, m = phi.shape

    w = np.random.rand(m, 1)  # Generate initial w

    for i in range(num_iterations):
        w -= coef * f_prime_ridge(phi, w, y, n, lambda1)

    # MSE for final interation
    final_mse = mse_function(phi.dot(w), y)

    return w, final_mse

In [12]:
# Find MSE for sklearn and gradient descent
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1)
ridge.fit(X_train, y_train.flatten())
y_hat = ridge.predict(X_train)
y_hat = y_hat.reshape(-1, 1)

sklearn_mse = mse_function(y_hat, y_train)
print(f'sklearn mse for ridge: {sklearn_mse}')

w, mse = gradient_descent_ridge(X_train, y_train, 10000, 0.0002, 1)
print(f'Gradient descent mse for ridge: {mse}')

sklearn mse for ridge: 0.03353159728443031
Gradient descent mse for ridge: 13.093493548870391


In [20]:
# identify best lambda value
values = np.logspace(-4, 1, 10) 

best_value = values[0] # initialize best value
best_mse = float('inf') # Store intiial mse as very high value

for lambda_value in values:
    # Get predicted w for lambda
    w, mse = gradient_descent_ridge(X_train, y_train, 10000, 0.02, lambda_value)
    y_hat = X_val.dot(w)

    # get MSE for predictede w
    val_mse = mse_function(y_hat, y_val)

    if best_mse > val_mse:
        best_mse = val_mse
        best_value = lambda_value 

print(f'Ridge: Best lambda value: {best_value} with mse: {best_mse}')

Ridge: Best lambda value: 0.001291549665014884 with mse: 0.07015376052301812


Elastic Net

In [14]:
def f_prime_elasticnet(phi, w, y, n, lambda1, lambda2):
    return (2/n) * phi.T.dot(phi.dot(w) - y) + lambda1*np.sign(w) + lambda2*(2*w)

def mse_function(y_pred, y):
    return np.mean((y_pred- y)**2)

def gradient_descent_elasticnet(phi, y, num_iterations, coef, lambda1, lambda2):
    n, m = phi.shape

    w = np.random.rand(m, 1)  # Generate initial w

    for i in range(num_iterations):
        w -= coef * f_prime_elasticnet(phi, w, y, n, lambda1, lambda2)

    # MSE for final interation
    final_mse = mse_function(phi.dot(w), y)

    return w, final_mse

In [15]:
# Find MSE for sklearn and gradient descent
from sklearn.linear_model import ElasticNet
elasticnet = ElasticNet(alpha=2, l1_ratio=.5)
elasticnet.fit(X_train, y_train.flatten())
y_hat = elasticnet.predict(X_train)
y_hat = y_hat.reshape(-1, 1)

sklearn_mse = mse_function(y_hat, y_train)
print(f'sklearn mse for Elastic Net: {sklearn_mse}')

w, mse = gradient_descent_elasticnet(X_train, y_train, 10000, 0.0002, 1, 1)
print(f'Gradient descent mse for Elastic Net: {mse}')

sklearn mse for Elastic Net: 23.66471886740332
Gradient descent mse for Elastic Net: 18.556567442547877


In [19]:
values = np.logspace(-4, 1, 10) 

best_lambda1_value = values[0] # initialize best value
best_lambda2_value = values[0]
best_mse = float('inf') # Store intiial mse as very high value

# Double iteration
for lambda1_value in values:
    for lambda2_value in values:
        # Train on training data
        w, mse = gradient_descent_elasticnet(X_train, y_train, 10000, 0.02, lambda1_value, lambda2_value)
        
        # get MSE for predicted w
        y_hat = X_val.dot(w)
        val_mse = mse_function(y_hat, y_val)

        if best_mse > mse:
            best_mse = mse
            best_lambda1_value = lambda1_value
            best_lambda2_value = lambda2_value 

print(f'Elastic Net: Best lambda1 value: {best_lambda1_value} and best lambda2 value: {best_lambda2_value} with mse: {best_mse}')

Elastic Net: Best lambda1 value: 0.0001 and best lambda2 value: 0.0001 with mse: 0.032383289169010836


d)
Elastic net is better than using only one constraint because it has an mse of around 0.03, while lasso and ridge have an mse of .06 and .07 respectively